See problem description for [day 5](https://adventofcode.com/2022/day/5)

In [1]:
# STRIPPING WHITESPACE ON THE LEFT is NOT USEFUL in this problem.
def get_lines(file_type='sample'):
    '''
    Read in the lines of today's sample/input file line by line. 
    Assumes the file is in folder called 'inputs/'
    
    Parameters
    ----------
    file_type : str
        Either sample or input
    
    Returns
    -------
    list of inputs stripped of whitespace
    '''
    import datetime
    day = str(datetime.datetime.today().day).zfill(2)
    filename = f'inputs/{day}-{file_type}.txt'
    try:
        with open(filename,'r') as file:
            lines = [line.rstrip() for line in file.readlines()]
        return lines
    except:
        print(filename+' does not exist')

In [2]:
config = get_lines()

In [3]:
config

['    [D]',
 '[N] [C]',
 '[Z] [M] [P]',
 ' 1   2   3',
 '',
 'move 1 from 2 to 1',
 'move 3 from 1 to 3',
 'move 2 from 2 to 1',
 'move 1 from 1 to 2']

First, identify the number of stacks: the first line in config that includes a number. Get the max number.

In [5]:
def get_number_of_stacks(config):
    for i, line in enumerate(config):
        for ch in line:
            if ch.isdigit():
                return (i, int(line[-1]))
    return -1

In [6]:
get_number_of_stacks(config)

(3, 3)

In [7]:
input_config = get_lines('input')

In [8]:
get_number_of_stacks(input_config)

(8, 9)

Next, get the value for a level of a stack. If it exists, it will be in location `1 + (stack_number-1)*4`. Stack numbering starts at 1!

In [9]:
labels, num_stacks = get_number_of_stacks(config)
labels, num_stacks

(3, 3)

In [19]:
def get_value(line, stack):
    try:
        ch = line[1+(stack-1)*4]
        if ch.isalpha():
            print(ch)
            return ch
        else:
            print('no value: before last valid')
            return None
    except:
        print('no value: after last valid')
        return None

In [20]:
get_value(config[0],1)

no value: before last valid


In [21]:
get_value(config[0],2)

D


'D'

In [22]:
get_value(config[0],3)

no value: after last valid


In [23]:
get_value(config[2],1)

Z


'Z'

In [24]:
from queue import LifoQueue
#LifoQueue behaves like a stack
# `put` a value onto a stack, `get` the top value removes the top value, `qsize` to get size of the stack

In [25]:
def initialize_state(config):
    labels, num_stacks = get_number_of_stacks(config)
    stacks = {i: LifoQueue() for i in range(1,num_stacks+1)}
    for i in range(labels-1,-1,-1):
        for stack in range(1, num_stacks+1):
            crate = get_value(config[i], stack)
#             print(crate)
            if crate is not None:
                stacks[stack].put(crate)
#         print('\n')
    return stacks

In [26]:
stacks = initialize_state(config)

Z
M
P
N
C
no value: after last valid
no value: before last valid
D
no value: after last valid


In [27]:
stacks

{1: <queue.LifoQueue at 0x1e7a6b68700>,
 2: <queue.LifoQueue at 0x1e7a6b68ca0>,
 3: <queue.LifoQueue at 0x1e7a6b68100>}

In [28]:
problem_stacks = initialize_state(get_lines('input'))

T
L
D
G
P
P
V
N
R
P
P
C
W
W
F
W
J
C
Z
T
Z
T
C
J
G
S
Q
C
V
F
D
no value: before last valid
D
B
Q
F
S
H
no value: before last valid
L
no value: before last valid
C
D
H
S
L
C
no value: before last valid
M
no value: before last valid
T
no value: before last valid
W
L
Q
G
no value: before last valid
V
no value: before last valid
S
no value: before last valid
no value: before last valid
V
N
no value: before last valid
no value: before last valid
C
no value: before last valid
Z
no value: after last valid
no value: after last valid
no value: after last valid


In [29]:
problem_stacks

{1: <queue.LifoQueue at 0x1e7a6b2a310>,
 2: <queue.LifoQueue at 0x1e7a6b686a0>,
 3: <queue.LifoQueue at 0x1e7a6b68d60>,
 4: <queue.LifoQueue at 0x1e7a6b68f40>,
 5: <queue.LifoQueue at 0x1e7a6b68040>,
 6: <queue.LifoQueue at 0x1e7a6b68130>,
 7: <queue.LifoQueue at 0x1e7a6b3c340>,
 8: <queue.LifoQueue at 0x1e7a6ba50a0>,
 9: <queue.LifoQueue at 0x1e7a6ba53a0>}

### Whew! Stacks have been initialized for inputs

### Next up...

Extrapolate the moves from the remainder of the file.

In [30]:
moves = config[labels+2:]
moves

['move 1 from 2 to 1',
 'move 3 from 1 to 3',
 'move 2 from 2 to 1',
 'move 1 from 1 to 2']

In [31]:
def extract_moves(move):
    extract = move.split(' ')
    num_to_move = int(extract[1])
    from_stack = int(extract[3])
    to_stack = int(extract[5])
    return (num_to_move, from_stack, to_stack)

extract_moves(moves[1])

(3, 1, 3)

In [32]:
for move in moves:
    print(extract_moves(move))

(1, 2, 1)
(3, 1, 3)
(2, 2, 1)
(1, 1, 2)


In [33]:
def check_stack_sizes(stacks):
    for stack in stacks:
        print(stack, stacks[stack].qsize())

In [34]:
check_stack_sizes(stacks)

1 2
2 3
3 1


In [35]:
check_stack_sizes(problem_stacks)

1 8
2 7
3 4
4 8
5 3
6 8
7 5
8 6
9 7


In [36]:
def make_move(stacks, move):
    num_to_move, from_stack, to_stack = extract_moves(move)
    for n in range(num_to_move):
        crate = stacks[from_stack].get()
        stacks[to_stack].put(crate)
        print(f'{crate} from stack {from_stack} to stack {to_stack}')
    return stacks

In [37]:
def make_moves(stacks, moves):
    for move in moves:
        stacks = make_move(stacks, move)
    return stacks

In [38]:
stacks = make_moves(stacks, moves)

D from stack 2 to stack 1
D from stack 1 to stack 3
N from stack 1 to stack 3
Z from stack 1 to stack 3
C from stack 2 to stack 1
M from stack 2 to stack 1
M from stack 1 to stack 2


In [39]:
check_stack_sizes(stacks)

1 1
2 1
3 4


In [43]:
def get_top_crates(stacks):
    '''
    Leave the state of the stacks alone! 
    '''
    answer = ''
    for stack in stacks:
        ch = stacks[stack].get()
        answer += ch
        stacks[stack].put(ch)
    return answer

In [44]:
get_top_crates(stacks)

'CMZ'

In [46]:
labels, num_stacks = get_number_of_stacks(input_config)

In [48]:
problem_moves = input_config[labels+2:]


In [49]:
answer_stacks = make_moves(problem_stacks, problem_moves)

G from stack 2 to stack 1
C from stack 2 to stack 1
H from stack 2 to stack 1
V from stack 2 to stack 1
T from stack 2 to stack 1
P from stack 2 to stack 1
Z from stack 6 to stack 3
S from stack 6 to stack 3
T from stack 6 to stack 3
C from stack 6 to stack 3
D from stack 6 to stack 5
C from stack 3 to stack 8
T from stack 3 to stack 8
S from stack 3 to stack 8
Z from stack 3 to stack 8
F from stack 3 to stack 8
Z from stack 3 to stack 8
C from stack 3 to stack 8
D from stack 3 to stack 8
D from stack 8 to stack 2
C from stack 8 to stack 2
Z from stack 8 to stack 2
F from stack 8 to stack 2
Z from stack 8 to stack 2
S from stack 8 to stack 2
T from stack 8 to stack 2
C from stack 8 to stack 2
W from stack 8 to stack 2
H from stack 8 to stack 2
Q from stack 8 to stack 2
S from stack 8 to stack 2
J from stack 8 to stack 2
D from stack 7 to stack 6
B from stack 7 to stack 6
P from stack 1 to stack 6
T from stack 1 to stack 6
V from stack 1 to stack 6
H from stack 1 to stack 6
C from stack

In [50]:
get_top_crates(answer_stacks)

'SVFDLGLWV'